In [19]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [20]:
from fastai.text import *

In [27]:
path = untar_data(URLs.IMDB_SAMPLE)
path

PosixPath('/home/jupyter/.fastai/data/imdb_sample')

In [28]:
df = pd.read_csv(path/'texts.csv', header=None)
example_text = df.iloc[2][1]; example_text

'This is a extremely well-made film. The acting, script and camera-work are all first-rate. The music is good, too, though it is mostly early in the film, when things are still relatively cheery. There are no really superstars in the cast, though several faces will be familiar. The entire cast does an excellent job with the script.<br /><br />But it is hard to watch, because there is no good end to a situation like the one presented. It is now fashionable to blame the British for setting Hindus and Muslims against each other, and then cruelly separating them into two countries. There is some merit in this view, but it\'s also true that no one forced Hindus and Muslims in the region to mistreat each other as they did around the time of partition. It seems more likely that the British simply saw the tensions between the religions and were clever enough to exploit them to their own ends.<br /><br />The result is that there is much cruelty and inhumanity in the situation and this is very u

In [29]:
tokenizer = Tokenizer()
tok = SpacyTokenizer('en')
' '.join(tokenizer.process_text(example_text, tok))

'xxmaj this is a extremely well - made film . xxmaj the acting , script and camera - work are all first - rate . xxmaj the music is good , too , though it is mostly early in the film , when things are still relatively cheery . xxmaj there are no really superstars in the cast , though several faces will be familiar . xxmaj the entire cast does an excellent job with the script . \n \n  xxmaj but it is hard to watch , because there is no good end to a situation like the one presented . xxmaj it is now fashionable to blame the xxmaj british for setting xxmaj hindus and xxmaj muslims against each other , and then cruelly separating them into two countries . xxmaj there is some merit in this view , but it \'s also true that no one forced xxmaj hindus and xxmaj muslims in the region to mistreat each other as they did around the time of partition . xxmaj it seems more likely that the xxmaj british simply saw the tensions between the religions and were clever enough to exploit them to their own

In [30]:
rm_useless_spaces("Inconsistent   use  of     spaces.")

'Inconsistent use of spaces.'

In [41]:
class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = []
        self.counter = collections.Counter()
        self.total = 0

    def add_word(self, word):
        if word not in self.word2idx:
            self.idx2word.append(word)
            self.word2idx[word] = len(self.idx2word) - 1

        token_id = self.word2idx[word]
        self.counter[token_id] += 1
        self.total += 1
        return token_id

    def __len__(self):
        return len(self.idx2word)


class Corpus(object):
    def __init__(self, path):
        self.dictionary = Dictionary()
        self.train = self.tokenize(os.path.join(path, 'train.txt'))
        self.valid = self.tokenize(os.path.join(path, 'valid.txt'))
        self.test = self.tokenize(os.path.join(path, 'test.txt'))
        self.num_tokens = len(self.dictionary)

    def tokenize(self, path):
        """Tokenizes a text file."""
        assert os.path.exists(path)
        # Add words to the dictionary
        with open(path, 'r') as f:
            tokens = 0
            for line in f:
                words = line.split() + ['<eos>']
                tokens += len(words)
                for word in words:
                    self.dictionary.add_word(word)

        # Tokenize file content
        with open(path, 'r') as f:
            ids = torch.LongTensor(tokens)
            token = 0
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    ids[token] = self.dictionary.word2idx[word]
                    token += 1

        return ids
    
# https://github.com/fastai/fastai_docs/blob/master/dev_nb/007_wikitext_2.ipynb
# download data from https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-v1.zip
EOS = '\n'
PATH=Path('')

def read_file(filename):
    tokens = []
    with open(PATH/filename, encoding='utf8') as f:
        for line in f:
            tokens.append(line.split() + [EOS])
    return np.array(tokens)

In [42]:
train_tok = read_file('train.txt')
valid_tok = read_file('valid.txt')
test_tok = read_file('test.txt')

# cor = Corpus('')
# voc = Vocab(cor.dictionary.idx2word)

In [34]:
a = list(train_tok)

In [35]:
flat_list = [item for sublist in a for item in sublist]

In [36]:
len(flat_list)
# 600k tokens
# * 2 ~ ptb
# * 5 ~ wikitext-2
# * 300 ~ wikitext-103

602169

In [15]:
valid_tok

array([list(['Dear', 'friends,', 'today', 'is', 'the', '13th', 'of', 'August,', '1996,', 'and', 'we', 'are', 'in', 'the', 'Upper', 'Hamlet.', 'We', 'are', 'going', 'to', 'speak', 'English.']),
       list([]),
       list(['The', 'other', 'day', 'after', 'I', 'spoke', 'about', 'the', 'practice', 'of', 'the', 'four', 'mantras.', 'I', 'said', 'that', 'the', 'fourth', 'mantra', 'is', 'more', 'difficult,', 'so', 'I', 'did', 'not', 'talk', 'about', 'it.', 'In', 'fact', 'it', 'is', 'difficult,', 'but', 'not', 'so', 'difficult.', 'After', 'the', 'Dharma', 'talk,', 'when', 'we', 'were', 'about', 'to', 'do', 'walking', 'meditation,', 'there', 'was', 'a', 'gentleman', 'who', 'stopped', 'me', 'on', 'my', 'way', 'and', 'asked', 'me', 'about', 'the', 'fourth', 'mantra.', 'He', 'was', 'very', 'eager', 'to', 'learn', 'and', 'to', 'practice', 'the', 'fourth', 'mantra.', 'He', 'was', 'very', 'curious,', 'because', 'I', 'had', 'said', 'that', 'the', 'fourth', 'mantra', 'is', 'more', 'difficult.', 'But',

In [42]:
36000*30

1080000

In [6]:
bs = 256
bptt = 35

In [25]:
data = (TextList.from_folder('')
            .split_by_files('valid.txt')
            .label_for_lm()           
            .databunch(bs=bs, num_workers=1))

In [43]:
dat = TextLMDataBunch.from_tokens('data', 
                                  trn_tok = train_tok, trn_lbls = None,
                                  val_tok = valid_tok, val_lbls=None,
                                  tst_tok = test_tok, 
#                                   vocab = voc,
                                  bs = bs, bptt = bptt)

In [26]:
?data

Type:        TextLMDataBunch
String form:
TextLMDataBunch;
           
           Train: LabelList (4 items)
           x: LMTextList
           xxbos xxunk xxmaj contents
           xxmaj no <...>  the questions of training and xxmaj sangha building .
           
           
           y: LMLabelList
           ,
           Path: .;
           
           Test: None
File:        /opt/anaconda3/lib/python3.7/site-packages/fastai/text/data.py
Docstring:   Create a `TextDataBunch` suitable for training a language model.


In [8]:
# [] list has some method
# voc.itos is a list so it has those default method

In [51]:
dat.train_ds

LabelList (58665 items)
x: LMTextList

,
,
,
,

y: LMLabelList
,,,,
Path: data

In [28]:
# need to avoid using \n when not end of line
data.show_batch()

idx,text
0,"xxmaj when we ’re eating , we \n know that we are eating . xxmaj when we open a \n door , we know that we ’re opening a door . xxmaj our \n mind is with our actions . \n xxmaj when you put a piece of fruit into your \n mouth , all you need is a little bit of mindfulness \n"
1,"’ll have joy . \n xxup xxunk \n \n \n xxmaj the xxmaj planet xxmaj is xxmaj us \n xxmaj our food comes from this beautiful planet . xxmaj the \n xxmaj earth is inside of us , in each morsel of food , in \n the air we breathe , in the water that we drink \n and that flows through us ."
2,"to be shared with young \n people before a meal , but they can be enjoyed \n by xxunk . \n 1 . this food is the gift of the whole universe : \n the xxmaj earth , the sky , the rain , and the sun . \n 2 . xxup w e thank the people who have made this \n food ,"
3,"health , in the xxunk incidence of heart attacks , \n cancer , and other fatal diseases . xxmaj the average person \n looking out on this ever - changing , seemingly xxunk \n world sees anything but natural karmic laws at work , \n nor does he perceive the unity and harmony under- \n lying this constant and inevitable change . xxmaj if anything"
4,"xxmaj using \n \n \n xxup awareness xxup of xxup being 21 \n the telephone , i wish that all living beings should free \n themselves of doubt and prejudice in order that com- \n munication between them should be readily estab- \n lished . "" \n xxmaj when i was seventeen years xxunk , i thought that the \n xxmaj little xxmaj"


In [44]:
learn = language_model_learner(data, AWD_LSTM, drop_mult=0.3)

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot(skip_end=15)

In [45]:
learn.fit_one_cycle(1, 1e-2, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,3.899552,3.569257,0.295536,01:40


In [46]:
learn.unfreeze()

In [47]:
learn.fit_one_cycle(2, 1e-3, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,3.399759,3.439830,0.318359,02:06
1,3.191141,3.534486,0.279074,02:07


In [53]:
learn.export('../viriya_deploy/tnh.pkl')

In [48]:
TEXT = "Practicing Concentration"
N_WORDS = 100
N_SENTENCES = 5

In [49]:
print(learn.beam_search('Anger is good.', n_words=20, beam_sz=20))

Anger is good. Anger is good . It is a 
  good example . It is a good way to be able to be 
 


In [50]:
print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

Practicing Concentration Mindfulness Training 
  While not practicing or practicing meditation , Bodhisattva ’s Friend 
  Mindfulness Training 
  In the United States , the First Mindfulness Training ( U.S. ) is an occasion for a meditation 
  on Awakening . There are 
  several books in the American Buddhist 
  Order of the Holy Spirit , such as Buddhist Meditation , Mindfulness and Meditation , and First Mindfulness Training , you have
Practicing Concentration , Meditation on Walking , and Meditation on the Nature of Mindfulness . 
  The Five Factors of Awakening 
  The Five Factors of Awakening 
  The Three Factors of Awakening 
  The Five Realizations of Atman 
  The first three Trainings were preached by Living Beings in the Fifth 
  Mindfulness Training . The second part , of course , vision of the 
  Four Foundations of
Practicing Concentration 
  The CONTINUING CONCENTRATION BREATH and Concentration Training ( SET ) , Mindfulness Training ( Follow the Teachings of The Great Bei

Using from folders

In [39]:
print(learn.beam_search('Anger is good.', n_words=20, beam_sz=20))

Anger is good. Anger is good . It is a 
  very simple , simple , and simple way of living . It is a


In [40]:
print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

Practicing Concentration 
  In the West , Genetic Awakening , The Way of Enlightenment 
  Every year , the Buddha delivers the Eight Realizations of Awakening . The Sutra on the Sangha of Mindfulness , a 
  box set of five full - length four - legged chairs is also available at the Center for the Present Moment . It contains the Sutra on Mindfulness , the 
  Complete Sutra on the Five Gates of
Practicing Concentration Knots , Noble Truths , and Noble Truths . 
  The first equivalents of Dharma talk are Dharma talks and Reconciliation Talks from Buddhist monks . These 
  are mostly reserved for us . The first is a Buddhist teaching . This 
  is the first full - length talk of the Dharma . The first is 
  the Dharma talk . First , 
  the second training is to sit down and give the Dharma to the community .
Practicing Concentration Life 
  The Five Practices of Mindfulness Training 
  Training for Training 
  Training training 
  Training is a training for training . In the training , tra

In [37]:
print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

Anger Chapter West - said, words. The Buddha told the Buddha that the Buddha had preparing them for a special meal in the future. The Buddha said, Only our friend King Bimbisara is a great All a spiritual Rahula, and the senior bhikkhu in his nation would be the Buddha’s The
Anger you are a long time in your daily heart. If you are fully awakened and at that moment you are suddenly the same or Buddha, I will never forget that you are I take the time to tell you about the Buddha in the very nature of his own body,


In [3]:
import urllib.request


In [6]:
...
# Download the file from `url` and save it locally under `file_name`:
urllib.request.urlretrieve(url='https://drive.google.com/uc?export=download&id=1aONdl2-puQYv6LylrPolIWKE269kj6mL')

('/tmp/tmpm4gg0br_', <http.client.HTTPMessage at 0x7fd91b4d4eb8>)

In [7]:
import urllib.request
import shutil

In [9]:

# Download the file from `url` and save it locally under `file_name`:
with urllib.request.urlopen(url='https://drive.google.com/uc?export=download&id=1aONdl2-puQYv6LylrPolIWKE269kj6mL') as response, open('t.pkl', 'wb') as out_file:
    shutil.copyfileobj(response, out_file)

In [4]:
?urllib.request.urlretrieve

Signature: urllib.request.urlretrieve(url, filename=None, reporthook=None, data=None)
Docstring:
Retrieve a URL into a temporary location on disk.

Requires a URL argument. If a filename is passed, it is used as
the temporary file location. The reporthook argument should be
a callable that accepts a block number, a read size, and the
total file size of the URL target. The data argument should be
valid URL encoded data.

If a filename is passed and the URL points to a local resource,
the result is a copy from local file to new file.

Returns a tuple containing the path to the newly created
data file as well as the resulting HTTPMessage object.
File:      /opt/anaconda3/lib/python3.7/urllib/request.py
Type:      function


In [ ]:

html = response.read()